In [6]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import nltk
import re
from nltk.stem import PorterStemmer
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn.metrics import classification_report

In [67]:
file = open("train.txt", "r", encoding="utf-8")
data = file.readlines()

data = [i.split("\t") for i in data]

df = [[row[2], row[4]] for row in data]  # Extract 2nd and 4th elements
df = pd.DataFrame(df, columns = ['genre', 'plot'])
print(df.head())


0       Sekhar (Tarun) is a graduate from IIM and work...
1       Kris Fowles (Katie Cassidy) goes to the Spring...
2       Cynthia is traumatized by the death of her bab...
3       Four friends, Gangu (Jackie Shroff), Abdul (Na...
4       Crisis in a middle-class family when the son f...
                              ...                        
8036    Dianne seems to have a happy family. She and h...
8037    Three lifelong friends work the bars in 1980's...
8038    City-based Ashok, who works as a Salesman in a...
8039    The story starts with narration by Maddy (R. M...
8040    In the British city of Everytown, businessman ...
Name: plot, Length: 8041, dtype: object


In [69]:
porter_stemmer=PorterStemmer()
def processSentence(s):
    words=re.split("\\s+",s)
    stemmed_words=[porter_stemmer.stem(word=w) for w in words]
    return ' '.join(stemmed_words)

x_processed = [processSentence(x) for x in df["plot"]]

print(x_processed[0])


sekhar (tarun) is a graduat from iim and work as a busi manag in a firm in hyderabad. he meet amar varma (siva balaji) in a busi parti who run a compani name varma industri in germany. when the parti is over, sekhar accident kill amar with hi car. sekhar' friend sunil (sunil) save him by eras the evidence, but sekhar feel guilti even after six months. he goe to germani along with sunil to visit amar' family. they find that their busi is in a bad condit and want to help them by reviv it with hi busi skills. priya (shriya), who wa engag to amar earlier is tri her level best to run the business, but helpless to run it smoothly. sekhar introduc himself to priya and offer hi help to reviv the business. the remain stori is about how sekhar help the busi grow and win priya' heart in that process. 
